In [ ]:
!pip install numpy tensorflow

In [ ]:
import random
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Activation
from tensorflow.keras.optimizers import RMSprop

In [ ]:
filepath = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [ ]:
text = open(filepath, 'rb').read().decode(encoding='utf-8').lower()
text = text[300000:800000]

In [ ]:
characters = sorted(set(text))
char_to_index = {c: i for i, c in enumerate(characters)}
index_to_char = {i: c for i, c in enumerate(characters)}

In [ ]:
SEQ_LENGTH = 40
STEP_SIZE = 3

sentences = [] #features
next_characters = [] # targets

In [ ]:
for i in range(0, len(text) -SEQ_LENGTH, STEP_SIZE):
    sentences.append(text[i: i+SEQ_LENGTH])
    next_characters.append(text[i+SEQ_LENGTH])

In [ ]:
x = np.zeros((len(sentences), SEQ_LENGTH, len(characters)))
y = np.zeros((len(sentences), len(characters)))

In [ ]:
# One-hot encode the input and output sequences
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_to_index[char]] = 1  # One-hot encoding for input
    y[i, char_to_index[next_characters[i]]] = 1  # One-hot encoding for output (next character)

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH, len(characters))))  # LSTM layer with 128 units
model.add(Dense(len(characters)))  # Output layer with units equal to the number of characters
model.add(Activation('softmax'))  # Softmax activation for multi-class classification

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(learning_rate=0.01))
model.fit(x, y, batch_size=256, epochs=4)

Epoch 1/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 120s 181ms/step - loss: 2.4785
Epoch 2/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 140s 178ms/step - loss: 1.7714
Epoch 3/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 143s 180ms/step - loss: 1.6119
Epoch 4/4
651/651 ━━━━━━━━━━━━━━━━━━━━ 121s 185ms/step - loss: 1.5259


In [16]:
def sample(preds, temprature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temprature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
def generate_text(length, temperature):
    start_index = random.randint(0, len(text) - SEQ_LENGTH - 1)
    generated = ''
    sentence = text[start_index: start_index + SEQ_LENGTH]
    generated += sentence
    for i in range(length):
        x = np.zeros((1, SEQ_LENGTH, len(characters)))
        for t, character in enumerate(sentence):
             x[0, t, char_to_index[character]] = 1
        predictions = model.predict(x, verbose=0)[0]
        next_index = sample(predictions, temperature)
        next_character = index_to_char[next_index]

        generated += next_character
        sentence = sentence[1:] + next_character
    return generated

In [ ]:
print('---------0.2-----------')
print(generate_text(300, 0.2))
print('---------0.4-----------')
print(generate_text(300, 0.4))
print('---------0.6-----------')
print(generate_text(300, 0.6))
print('---------0.8-----------')
print(generate_text(300, 0.8))
print('---------1.0-----------')
print(generate_text(300, 1.0))

---------0.2-----------
elcome is the stroke of death to me
than the heart the grace the heart the grace
and the heart of the shall i am the streads,
the string the forth the heart of the grief
and the come in the heart the heart of the son
the forth the heaven the world and men the grace
and the the fout the heart with the many and the grace
and the tongue the 
---------0.4-----------
hou fortunate!

montague:
comfort, my lord, and there see the gentlemen
shall the world and be sett the days the worther
the would be the forth which fortune contence
and the more and we come stand the counsers
and the tongue the bentle this blood, there streach,
and i am my most him to the secr;
and my son in the world to the heart and t
---------0.6-----------
ll her, emilia.
i'll use that tongue i hate temple foreft
and there is the eyes of the trangent to the sire,
but this shall be men the rest fortune and longeed,
and num best the crown shall peace to the clarence,
by the heaven not you to can my f